<a href="https://colab.research.google.com/github/inf032/introducaoML/blob/master/Farm02.Comparacao_Classificadores_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação de Bibliotecas**


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics  import mean_squared_error
from sklearn.dummy import DummyRegressor

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
from sklearn.impute._base import _BaseImputer




**Classe para imputação e coluna de missing**

In [16]:
class ImputerWithMissingMark (BaseEstimator, TransformerMixin):

    def transformColunas_IMPUTED(self, X) -> pd.DataFrame :
      copia = X.copy()
      for col in self.col_name_with_nan :
          copia[f'{col}{self.postfix}'] = copia[col].isna().astype(int)
      return copia      

    def col_with_nan(self, X) -> list:
      list_col_with_nan = []
      copia = X.copy()
      for col in copia.columns :
        if(copia[col].isna().sum() > 0) :
          list_col_with_nan.append(col)
      return list_col_with_nan

    def col_num_with_nan(self, X) -> list:
      cols = X.shape[1]
      list_col_with_nan = []
      for col in range(cols):    
          if(np.count_nonzero(np.isnan(X[:,col])) > 0) :  
              list_col_with_nan.append(col)
      return list_col_with_nan        

    def __init__(self, imputer=KNNImputer(), postfix = '_IMPUTED'):
      self.imputer = imputer
      self.col_name = None
      self.fitted = False
      self.col_name_with_nan = None
      self.col_number_with_nan = None
      self.postfix = postfix

    def fit_DF(self, X):
      self.col_name = X.columns.values
      self.col_name_with_nan = self.col_with_nan(X)

    def fit_NP(self, X):
      self.col_number_with_nan = self.col_num_with_nan(X)

    def transform_DF(self, X):
      copia = self.transformColunas_IMPUTED(X)
      transformed = pd.DataFrame(self.imputer.transform(copia[self.col_name]))
      transformed.columns = X.columns
      transformed.index = X.index
      for col in self.col_name_with_nan :
        transformed[f'{col}{self.postfix}'] = copia[f'{col}{self.postfix}']
      return transformed

    def transform_NP(self, X):
      transformed = np.copy(X)
      vazio = np.zeros(shape=(X.shape[0],0))
      for col in self.col_number_with_nan:
        a_nan = [1 if np.isnan(x) else 0 for x in transformed[:,col]]
        vazio = np.append(vazio, np.c_[a_nan], axis = 1)
      transformed = self.imputer.transform(transformed)  
      for col in range(vazio.shape[1]) :
        transformed = np.append(transformed, np.c_[vazio[:, col]], axis = 1)
      return transformed      

    def fit(self, X, y=None):
      if(self.fitted == False) :
        self.fitted = True
        if(type(X) == pd.DataFrame) :
          self.fit_DF(X)
        else :
          self.fit_NP(X)  
        self.imputer.fit(X)
      return self

    def transform(self, X, y=None):
      if(self.fitted == True) :
        if(self.col_name is not None) :
          transformed = self.transform_DF(X)    
        else :
          transformed = self.transform_NP(X)    
        return transformed
      else :
        return X

    def get_params(self, deep=True):
      # retorna um dicionário com os atributos e seus respectivos valores
      return {"imputer": self.imputer, 
              "postfix": self.postfix}
      
    def set_params(self, **parameters):
      # iterando pelos elementos passados e definindo-os na classe
      for parameter, value in parameters.items():
        setattr(self, parameter, value)
      return self 

**Função para impressão de desempenho de teste e treino baseado na acurácia**

In [17]:
def report_score(title, model, X_train, y_train, X_test, y_test) :
  model.fit(X_train, y_train)
  print(title, '\n')
  print(f'Acurácia treino: {np.round(model.score(X_train, y_train) *100, 2)}%')
  print(f'Acurácia teste: {np.round(model.score(X_test, y_test) *100, 2)}%')

**Criação de Tipos diferentes de classificadores, que serão usados na comparação**

In [18]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier




nomes = ['Dummy', 'Ridge', 'SVM', 'KNN', 
         'Gaussian Process',
         'Gaussian Naives Bayes', 'Decision Tree']

classificadores = [DummyClassifier(strategy='stratified'),
                   RidgeClassifier(),
                   SVC(),
                   KNeighborsClassifier(),
                   GaussianProcessClassifier(),
                   GaussianNB(),
                   DecisionTreeClassifier()]

**Leitura dos Dados e Holdout**


In [19]:
SEED = 1234
np.random.seed(SEED)
warnings.filterwarnings(action='once')

df = pd.read_csv('https://raw.githubusercontent.com/inf032/introducaoML/master/data/farmaco.csv')
df.columns = ['IDADE', 'SEXO', 'PA', 'COLESTEROL', 'NA_K', 'FARMACO']
preditoras = ['IDADE', 'SEXO', 'PA', 'COLESTEROL', 'NA_K']
alvo = 'FARMACO'

X = df[preditoras]
y = df[alvo]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=SEED)

print(f'Holdout: {X_train.shape[0]} individuos em treino. {X_test.shape[0]} individuos em teste.')

Holdout: 157 individuos em treino. 68 individuos em teste.


**Criação do Pipeline**

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

ordinal_transformer = Pipeline(steps=[('ordinal_encoder', OrdinalEncoder(categories=[['LOW', 'NORMAL', 'HIGH'], 
                                                                                     ['LOW', 'NORMAL', 'HIGH']], 
                                                                          handle_unknown='use_encoded_value',
                                                                          unknown_value=np.nan))
                                     ])
onehot_transformer = Pipeline(steps=[('one_hot_encoder',  OneHotEncoder(drop='if_binary', sparse=False))])


col_transformer = ColumnTransformer(transformers=[('pa_colesterol', ordinal_transformer, ['PA', 'COLESTEROL']),
                                                  ('sex', onehot_transformer, ['SEXO'])],
                                    remainder='passthrough')

**Avaliação do Score dos Classificadores**


In [21]:
resultados = []

for nome, model in zip(nomes, classificadores) :
    score = Pipeline(steps=[('step_one', col_transformer), ('ss',  StandardScaler()), 
                           ('imputer', ImputerWithMissingMark()), ('classificador', model)]).fit(X_train, y_train).score(X_test, y_test)
    resultados.append([nome, np.round(score*100,2)])
    
dfResultados = pd.DataFrame(resultados, columns = ['CLASSIFICADOR', 'ACURACIA'])

dfResultados


,CLASSIFICADOR,ACURACIA
0,Dummy,50.00
1,Ridge,86.76
2,SVM,95.59
3,KNN,85.29
4,Gaussian Process,89.71
5,Gaussian Naives Bayes,86.76
6,Decision Tree,97.06


**Gráfico com a comparação dos Valores de Acurácia**




In [22]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

fig = go.Figure()
fig = px.scatter(dfResultados, x='CLASSIFICADOR', y='ACURACIA')
fig.update_layout(
    title="Acurácia por Modelo",
    xaxis_title="Classificador",
    yaxis_title="Acurácia",
    font=dict(
        family="Verdana",
        size=16,
        color="RebeccaPurple"
    )
)
fig.show()

**Avaliação do Score dos Classificadores com Validação Cruzada (IC: 95%)**

In [23]:
from sklearn.model_selection import cross_validate

resultados_cv = []

for nome, model in zip(nomes, classificadores) :
    modelo = Pipeline(steps=[('step_one', col_transformer), ('ss',  StandardScaler()), 
                           ('imputer', ImputerWithMissingMark()), ('classificador', model)])
    resultado_cv = cross_validate(modelo, X_train, y_train, cv=5)
    resultados_cv.append([nome, np.round(resultado_cv['test_score'].mean()*100,2), np.round(resultado_cv['test_score'].std()*100,2)])
    
dfResultados_cv = pd.DataFrame(resultados_cv, columns = ['CLASSIFICADOR', 'ACURACIA_MEDIA', 'ACURACIA_STD'])

dfResultados_cv

,CLASSIFICADOR,ACURACIA_MEDIA,ACURACIA_STD
0,Dummy,54.80,9.08
1,Ridge,86.61,5.16
2,SVM,89.21,6.43
3,KNN,80.99,9.34
4,Gaussian Process,87.96,7.20
5,Gaussian Naives Bayes,75.24,14.07
6,Decision Tree,96.81,2.04


**Gráfico com a comparação dos Valores de Acurácia (IC: 95%)**

In [24]:
fig = px.scatter(dfResultados_cv, x='CLASSIFICADOR', y='ACURACIA_MEDIA', 
                 error_y='ACURACIA_STD')
fig.update_layout(
    title="Acurácia por Modelo",
    xaxis_title="Classificador",
    yaxis_title="Acurácia",
    font=dict(
        family="Verdana",
        size=16,
        color="RebeccaPurple"
    )
)
fig.show()